In [1]:
from gurobipy import *
from prettytable import PrettyTable
import numpy as np

In [2]:
OPF = Model('2 bus OPF')

In [3]:
#Sets
G = 2 # Generators
B = 2 # Bus
L = 1 #Lines
Gen = range(G)
Bus = range(B)
Line = range(L)

In [4]:
L2= 400 # electircal Load (MW)
X12 = 0.2 # Reactance (ohms)
Sbase = 100 # Power Base (MVA)
P12_max = 1.5 

In [5]:
GenAt = {Gen[0]:{'a':3 ,'b':20 ,'c':100, 'Pmin':28 ,'Pmax':206},
         Gen[1]:{'a':4.05 ,'b':18.07 ,'c':98.87, 'Pmin':90 ,'Pmax':284}}

In [6]:
Pmin = [(GenAt[i]['Pmin'])/Sbase for i in Gen]
Pmax = [(GenAt[i]['Pmax'])/Sbase for i in Gen]
g = [a for a in Gen]
b = [a for a in Bus]
l = [a for a in Line]

In [7]:
P = OPF.addVars(g, lb = Pmin, ub = Pmax, name = 'Gen')
P12 = OPF.addVars(l, lb = -P12_max, ub = P12_max, name = 'line')
Delta = OPF.addVars(b, lb = -6, ub = 6, name = 'PhaseAngle')
P12[0]

<gurobi.Var *Awaiting Model Update*>

In [8]:
Genline = OPF.addConstr(P[0] == P12[0], name ='GenLine')
lineFLow = OPF.addConstr(P[1] +P12[0] == L2/Sbase, name = 'lineFlow')
BusAngle = OPF.addConstr(P12[0] == (Delta[0]-Delta[1])/X12, name = 'BusAngle')
DeltaConstr = OPF.addConstr(Delta[0] == 0, name = 'DeltaConstr')

In [9]:
OF = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)

In [10]:
OPF.setObjective(OF, GRB.MINIMIZE)
OPF.Params.Method=-1
OPF.setParam( 'OutputFlag', True )
OPF.optimize()



status = OPF.status
if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE \
or status == GRB.Status.UNBOUNDED:
    OPF.computeIIS()
    OPF.write("model.ilp")

Parameter Method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 4 rows, 5 columns and 8 nonzeros
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [2e+01, 2e+01]
  QObjective range [6e+00, 8e+00]
  Bounds range     [3e-01, 6e+00]
  RHS range        [4e+00, 4e+00]
Presolve removed 3 rows and 3 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.57755765e+05 -4.58065460e+05  2.00e+03 5.06e+02  1.00e+06     0s
   1   3.23963236e+02 -1.22489368e+04  2.08e+00 5.26e-01  

In [11]:
OPF.printStats()


Statistics for model 2 bus OPF :
  Linear constraint matrix    : 4 Constrs, 5 Vars, 8 NZs
  Matrix coefficient range    : [ 1, 5 ]
  Objective coefficient range : [ 18.07, 20 ]
  Variable bound range        : [ 0.28, 6 ]
  RHS coefficient range       : [ 4, 4 ]


In [12]:
print(Pmin)
print(Pmax)

[0.28, 0.9]
[2.06, 2.84]


In [13]:
print(Delta[1])

<gurobi.Var PhaseAngle[1] (value -0.2999999999989068)>
